In [1]:
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from urllib.parse import parse_qs, urlparse
from datetime import datetime
import time
import random

# I. Crawl Information

Helper function to extract information about Hong Kong Bar Association - junior counsel

In [2]:
def extract_data(soup, language = 'Eng'):
    '''
    @soup: BeautifulSoup object contains html of the website
    @language: a string indicates whether the language is Chi or Eng
    
    the function extract member information on current page and return a data dictionary with below format
    return data: {'Member': name, 'Name of Chambers': ,Address': , 'Tel.No': , 'Fax.No':, 'Website': , Call':}
    ''' 
    data = {}
    
    # name format: last name Eng, middle name Eng, initials Eng, title, name Chi, title Chi 
    name = soup.find('div',{'class':'barrister-name'}).text.strip().split(' ')
    name = ' '.join(name[:-1]).strip()
    data['Member'] = name
    
    # other info, split key and value using the first :
    info = soup.find_all('div',{'class':'barrister-row'})
    for item in info:
        content = item.text
        key = content[:content.find(':')]
        value = content[content.find(':')+1:]
        data[key] = value
        
    return data

## English version 

Extract main page html, the website stores the names in javascript dataTable

In [3]:
eng_url = "https://www.hkba.org/Bar-List/junior-counsel"
session = requests.Session()
eng_html = session.get(eng_url)
soup = BeautifulSoup(eng_html.text, 'html.parser')

get the html where the javascript defines the dataSet variable

In [4]:
content = soup.find_all('script')[27].text
end_str = 'brian-yw","",""]'
idx = content.find(end_str)
# get only dataset html
content = content[:idx+len(end_str)]
# get only lists of html
content = content.split('[')[2:]

Get the html of each member's website

In [5]:
def extract_member_html(html):
    html_prefix = 'https://www.hkba.org/'
    url = re.search('barrister/[0-9a-zA-ZÀ-ÿ-.\-]+',html)
    if url:
        return html_prefix + url.group()
    else:
        return ""

In [6]:
content_df = pd.DataFrame(content, columns = ['html'])
content_df.html = content_df.html.apply(extract_member_html)
member_url = content_df.html.tolist()

Extract member data on all pages. To mimic human visits, set to headers as below.

In [7]:
USER_AGENT_LIST = [
    "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
    "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
    "Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)",
    "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
    "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
    "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
    "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0",
    "Mozilla/5.0 (X11; Linux i686; U;) Gecko/20070322 Kazehakase/0.4.5",
    "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.8) Gecko Fedora/1.9.0.8-1.fc10 Kazehakase/0.5.6",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20",
    "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; fr) Presto/2.9.168 Version/11.52",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.11 TaoBrowser/2.0 Safari/536.11",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.71 Safari/537.1 LBBROWSER",
    "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; LBBROWSER)",
    "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E; LBBROWSER)",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.84 Safari/535.11 LBBROWSER",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E)",
    "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; QQBrowser/7.0.3698.400)",
    "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; SV1; QQDownload 732; .NET4.0C; .NET4.0E; 360SE)",
    "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E)",
    "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.89 Safari/537.1",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.89 Safari/537.1",
    "Mozilla/5.0 (iPad; U; CPU OS 4_2_1 like Mac OS X; zh-cn) AppleWebKit/533.17.9 (KHTML, like Gecko) Version/5.0.2 Mobile/8C148 Safari/6533.18.5",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:2.0b13pre) Gecko/20110307 Firefox/4.0b13pre",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:16.0) Gecko/20100101 Firefox/16.0",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11",
    "Mozilla/5.0 (X11; U; Linux x86_64; zh-CN; rv:1.9.2.10) Gecko/20100922 Ubuntu/10.10 (maverick) Firefox/3.6.10",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36",
    ]

In [98]:
data = []
i = 0
start = time.time()
for url in member_url:
    USER_AGENT = random.choice(USER_AGENT_LIST)
    headers = {'user-agent': USER_AGENT}
    html = session.get(url, headers=headers)
    soup = BeautifulSoup(html.text, 'html.parser')
    data.append(extract_data(soup, 'Eng'))
    print(i, data[-1])
    i += 1
end = time.time()
print('Take {} second to crawl {} items'.format(end-start,len(member_url)))

0 {'Member': 'Wong, Pak-Heung', 'Address': '3/F, 50 Wyndham St,  Central, H.K.', 'Tel.No': '25233450', 'Fax.No': '-', 'Call': 'HK( 1966 ), UK( 1966  )'}
1 {'Member': 'Tam, Maria W.C. (Ms) 譚惠珠  (女士)', 'Name of Chambers': 'Kingston Chambers', 'Address': 'Unit 1203, 12/F, Bank of America Tower, 12 Harcourt Rd, Central, H.K.', 'Tel.No': '28698399', 'Fax.No': '25378363', 'Call': 'HK( 1974 ), UK( 1972  )'}
2 {'Member': 'Necholas, John 黎約翰', 'Name of Chambers': 'Garden Chambers', 'Address': '15/F, Printing House, 6 Duddell St, Central, H.K.', 'Tel.No': '25250221', 'Fax.No': '28452441', 'E-mail': 'admin@gardenchambers.com.hk', 'Call': 'HK( 1975 ), UK( 1972  )'}
3 {'Member': 'Mok, Yeuk-Chi 莫若志', 'Name of Chambers': "Patrick Yu's Chambers", 'Address': 'Rms 45-46, New Henry House, 10 Ice House St, Central, H.K.', 'Tel.No': '25257295', 'Fax.No': '28106239', 'Call': 'HK( 1975  )'}
4 {'Member': 'Yuen, Henry M. 袁文超', 'Address': 'Rm 3406 & Rm 412, Tower Two, Lippo Centre, 89 Queensway, Admiralty, H.K.

34 {'Member': 'Wong, Hay-Yiu 王熙曜', 'Address': '19/F, Yam Tze Commercial Bldg, 23 Thomson Road, Wanchai, H.K.', 'Tel.No': '25251229', 'Fax.No': '25279229', 'E-mail': 'hennessy138@biznetvigator.com', 'Call': 'HK( 1982 ), UK( 1981  )'}
35 {'Member': 'Hingorani, Jeevan 高華理', 'Name of Chambers': 'Pacific Chambers', 'Address': 'Rm 901, Dina House, 11 Duddell St, Central, H.K.', 'Tel.No': '25215544', 'Fax.No': '25245912', 'E-mail': 'jeevan@pacificchambers.com', 'Call': 'HK( 1982 ), UK( 1978  )'}
36 {'Member': 'Hemmings, John 夏明仕', 'Name of Chambers': 'South China Chambers', 'Address': 'Unit A, 18/F, China Overseas Bldg, 139 Hennessy Rd, Wanchai, H.K.', 'Tel.No': '25282378', 'Fax.No': '25201512', 'Mobile': '63413378', 'E-mail': 'johnhemmings.lawyer@gmail.com', 'Call': 'HK( 1982 ), UK( 1974  )'}
37 {'Member': 'Pinto, Josephine (Ms) 潘素安  (女士)', 'Name of Chambers': '601 Dina House', 'Address': 'Rm 601, Dina House, 11 Duddell St, Central, H.K.', 'Tel.No': '28452020', 'Fax.No': '25960769', 'Call': 

66 {'Member': 'Marray, John', 'Name of Chambers': 'Admiralty Chambers', 'Address': 'Rm 403, Fairmont House, 8 Cotton Tree Drive, Central, H.K.', 'Tel.No': '25273082', 'Fax.No': '25298226', 'Call': 'HK( 1985 ), UK( 1980  )'}
67 {'Member': 'Wong, Newman 王興偉 *', 'Address': '4/F, Chung Nam Bldg, 1 Lockhart Rd, Wanchai, H.K.', 'Tel.No': '25217317', 'Fax.No': '28450654', 'E-mail': 'bar1002@netvigator.com', 'Call': 'HK( 1986 ), UK( 1985  )'}
68 {'Member': 'Yuen, Bernard 袁國華', 'Name of Chambers': 'Academy Chambers', 'Address': 'Rm 1003, Far East Finance Centre, 16 Harcourt Rd, Admiralty, H.K.', 'Tel.No': '25255878', 'Fax.No': '25255877', 'E-mail': 'bernardyuen226@yahoo.com.hk', 'Quals': 'LLM (LSE) , LLB , PCLL (HKU)', 'Call': 'HK( 1986  )'}
69 {'Member': 'Mughal, Hanif M. 麥嘉豪', 'Name of Chambers': 'Garden Chambers', 'Address': '15/F, Printing House, 6 Duddell St, Central, H.K.', 'Tel.No': '25250221', 'Fax.No': '28452441', 'E-mail': 'admin@gardenchambers.com.hk', 'Call': 'HK( 1986 ), UK( 1985  

98 {'Member': 'Chan, Chi-Man 陳志民', 'Name of Chambers': 'Fortune Chambers', 'Address': 'Rm 2305-07, 23/F, Tower Two, Lippo Centre, 89 Queensway, Admiralty, H.K.', 'Tel.No': '25438099', 'Fax.No': '28452668', 'Call': 'HK( 1987 ), UK( 1987  )'}
99 {'Member': 'Raffell, Andrew J. 安華暉', 'Name of Chambers': 'Pacific Chambers', 'Address': 'Rm 901, Dina House, 11 Duddell St, Central, H.K.', 'Tel.No': '25215544', 'Fax.No': '25245912', 'E-mail': 'andrew.raffell@pacificchambers.com', 'Call': 'HK( 1987 ), UK( 1983  )'}
100 {'Member': 'Yip, Liza S.H. (Ms) 葉瑞紅  (女士) *', 'Name of Chambers': 'Tower Chambers', 'Address': "15/F, New World Tower 2, 18 Queen's Rd Central, H.K.", 'Tel.No': '25212555', 'Fax.No': '28106087', 'Call': 'HK( 1988 ), UK( 1987  )'}
101 {'Member': 'Shum, Edward P.S. 岑炳生', 'Address': '10/F, Printing House, 6 Duddell St, Central, H.K.', 'Tel.No': '25260068', 'Fax.No': '28400131', 'Quals': 'Solicitor (HK) [1991-1994] , Solicitor (UK) [1992-2004]', 'Call': 'HK( 1988  )(Voluntary Removal 

127 {'Member': 'Fung, Louis Karon 馮家安', 'Address': 'Rm 501, Yu Yuet Lai Bldg, 43-55 Wyndham Street, Central, H.K.', 'Tel.No': '28100173', 'Fax.No': '28100179', 'Call': 'HK( 1989  )'}
128 {'Member': 'So, Alan 蘇國強', 'Address': 'Rm 1003, South China Bldg, 1 Wyndham St, Central, H.K.', 'Tel.No': '25245677', 'Fax.No': '25243899', 'E-mail': 'alansochamber@gmail.com', 'Call': 'HK( 1989 ), UK( 1989  )'}
129 {'Member': 'Lam, Simon K.C. 藍建中 *', 'Name of Chambers': 'Fraternity Chambers', 'Address': 'Rm 1003A, 10/F, Far East Finance Centre, 16 Harcourt Rd, Admiralty, H.K.', 'Tel.No': '25298229', 'Fax.No': '25288383', 'E-mail': 'simonkclam@fraternity.hk', 'Quals': 'LLB , BSc , FHKI Arb , FCI Arb , MCS , HKMAAL Accredited General Mediator', 'Call': 'HK( 1989 ), UK( 1989  )'}
130 {'Member': 'Lin, Kenny C.P. 林清培', 'Name of Chambers': 'Tower Chambers', 'Address': "15/F, New World Tower 2, 18 Queen's Rd Central, H.K.", 'Tel.No': '25212555', 'Fax.No': '25375699', 'Call': 'HK( 1989 ), UK( 1989  )'}
131 {'

...

1291 {'Member': 'Tang, Alan S.L. 鄧兆麟 *', 'Name of Chambers': "Wong Man Kit SC's Chambers", 'Address': 'Suites 902-905, Bank of America Tower, 12 Harcourt Rd, Central, H.K.', 'Tel.No': '29189187', 'Fax.No': '29189197', 'Mobile': '69062822', 'E-mail': 'slatang1@gmail.com', 'Call': 'HK( 2018  )'}
1292 {'Member': 'Liu, Ka Pui Michelle (Ms) 劉嘉霈  (女士) *', 'Name of Chambers': 'Wyndham Chambers', 'Address': 'Rm 101-102, 1/F, Wilson House, 19-27 Wyndham St, Central, H.K.', 'Tel.No': '25302889', 'Fax.No': '28401396', 'Mobile': '66992208', 'E-mail': 'liukapui@hotmail.com', 'Quals': 'LLB (HKU) , PCLL (HKU)', 'Call': 'HK( 2018  )'}
1293 {'Member': 'Li, Pak Hei 李柏熹', 'Name of Chambers': "Sir Oswald Cheung's Chambers", 'Address': '10/F, New Henry House, 10 Ice House St, Central, H.K.', 'Tel.No': '25242156', 'Fax.No': '28105656', 'E-mail': 'lipakhei@siroswald.com', 'Call': 'HK( 2018  )'}
1294 {'Member': 'Mok, Cheuk Yuen Billy 莫倬原 *', 'Name of Chambers': 'Liberty Chambers', 'Address': '3102, Tower 1, A

1317 {'Member': 'Wong, Yeung Hou Howard 黃揚孝', 'Name of Chambers': 'Des Voeux Chambers', 'Address': '38/F, Gloucester Tower, The Landmark,  Central, H.K.', 'Tel.No': '25263071', 'Fax.No': '28105287', 'Call': 'HK( 2018  )'}
1318 {'Member': 'Lok, Tsz Kwan Jenny (Ms) 駱芷君  (女士)', 'Name of Chambers': 'David Ma’s Chambers', 'Address': "Rm 802, Loke Yew Bldg, 50 Queen's Rd Central, H.K.", 'Tel.No': '21539188', 'Fax.No': '21539187', 'Mobile': '69945000', 'E-mail': 'jtklok@gmail.com', 'Call': 'HK( 2018  )'}
1319 {'Member': 'Chan, Ka Yan Karen (Ms) 陳嘉恩  (女士) *', 'Name of Chambers': "Cheng Huan SC's Chambers", 'Address': '15/F, Dina House, 11 Duddell St, Central, H.K.', 'Tel.No': '25262293', 'Fax.No': '28453749', 'E-mail': 'kyk.chan@outlook.com', 'Call': 'HK( 2018  )'}
1320 {'Member': 'Au, Julia Ming-hao (Ms) 歐明皓  (女士)', 'Name of Chambers': "Sir Oswald Cheung's Chambers", 'Address': '10/F, New Henry House, 10 Ice House St, Central, H.K.', 'Tel.No': '25242156', 'Fax.No': '28105656', 'Call': 'HK( 20

1347 {'Member': 'Wong, Ying Kei Vivian (Ms) 黃纓淇  (女士)', 'Name of Chambers': "Cheng Huan SC's Chambers", 'Address': '15/F, Dina House, 11 Duddell St, Central, H.K.', 'Tel.No': '25262293', 'Fax.No': '28453749', 'Mobile': '60124861', 'E-mail': 'ykvwong@gmail.com', 'Call': 'HK( 2019  )'}
1348 {'Member': 'Wong, Timothy Ryan 王天 *', 'Name of Chambers': 'Equity Chambers', 'Address': 'Suite 802, 8/F, Harcourt House, 39 Gloucester Rd, Wanchai, H.K.', 'Tel.No': '21164676', 'Fax.No': '21164610', 'E-mail': 'twong@equitychambers.com.hk', 'Quals': 'BSc (HKUST) , JD (HKU) , LLM (LSE)', 'Call': 'HK( 2019  )'}
1349 {'Member': 'Ng, Edward Ka Hong 吳嘉康 *', 'Address': 'Rm 28, New Henry House, 10 Ice House St, Central, H.K. [Annex to Andrew Liao SC’s Chambers]', 'Tel.No': '35763130', 'Fax.No': '35763030', 'Mobile': '93427400', 'E-mail': 'mr.ng.edward@gmail.com', 'Quals': 'LLB (HKU) , PCLL (HKU) , LLM (LSE) , CEDR Accredited Mediator , MCIArb , AHKIArb', 'Call': 'HK( 2019  )'}
1350 {'Member': 'Tang, Ho Ling L

1378 {'Member': 'Chan, Leon Cheuk Hin 陳卓軒', 'Address': '10/F, Printing House, 6 Duddell St, Central, H.K.', 'Tel.No': '25260068', 'Fax.No': '28400131', 'Mobile': '98854187', 'E-mail': 'leonchchan@gmail.com', 'Quals': 'LLB (Soton) , MA (Human Rights) (UCL) , LLM (IT & IP) (HKU) , PCLL (HKU)', 'Call': 'HK( 2019  )'}
1379 {'Member': 'Daswani, Amrit', 'Name of Chambers': 'South China Chambers', 'Address': 'Unit A, 18/F, China Overseas Bldg, 139 Hennessy Rd, Wanchai, H.K.', 'Tel.No': '25282378', 'Fax.No': '25201512', 'E-mail': 'daswani.amrit@gmail.com', 'Quals': 'LLB , PCLL , CEDR Accredited Mediator', 'Call': 'HK( 2019  )'}
1380 {'Member': 'Tope, Joseph John 陶英傑', 'Name of Chambers': 'South China Chambers', 'Address': 'Unit A, 18/F, China Overseas Bldg, 139 Hennessy Rd, Wanchai, H.K.', 'Tel.No': '25282378', 'Fax.No': '25201512', 'Call': 'HK( 2019  )'}
1381 {'Member': 'Lau, Ka Yu Martin 劉加譽', 'Name of Chambers': 'Des Voeux Chambers', 'Address': '38/F, Gloucester Tower, The Landmark,  Centra

1406 {'Member': 'Tung, Yat Ting Ingrid (Ms) 董逸婷  (女士)', 'Name of Chambers': 'Admiralty Chambers', 'Address': 'Rm 403, Fairmont House, 8 Cotton Tree Drive, Central, H.K.', 'Tel.No': '25273082', 'Fax.No': '25298226', 'Call': 'HK( 2019  )'}
1407 {'Member': 'Lee, Tin Ho 李天浩', 'Name of Chambers': 'DES Chambers', 'Address': 'Unit C, 5/F, Eton Bldg, No.288 Des Voeux Rd Central, H.K.', 'Tel.No': '22433088', 'Fax.No': '22433077', 'Call': 'HK( 2019  )'}
1408 {'Member': 'Chan, Hei Ting Sheera (Ms) 陳希婷  (女士)', 'Address': 'Rm 87, New Henry House, 10 Ice House St, Central, H.K.', 'Tel.No': '25225494', 'Fax.No': '28104677', 'Call': 'HK( 2019  )'}
1409 {'Member': 'Pao, Peggy Pak Yiu (Ms) 包珀瑤  (女士) *', 'Name of Chambers': 'Pacific Chambers', 'Address': 'c/o Rm 901, Dina House, 11 Duddell St, Central, H.K.', 'Tel.No': '25215544', 'Fax.No': '25245912', 'Mobile': '64379355', 'E-mail': 'peggypypao@gmail.com', 'Call': 'HK( 2019  )'}
1410 {'Member': 'Tsang, Raymond W.N. 曾蔚南 *', 'Name of Chambers': "Alan Leon

Save data to Dataframe

In [99]:
df = pd.DataFrame(data)
df['Category'] = 'Professional Qualifications'
df['Organization'] = 'Hong Kong Bar Association'
df['Title'] = 'Junior Counsel'
df.to_csv('香港大律师公会_JuniorCounsel_raw.csv',encoding='utf_8_sig')
print(df.head())
print(df.shape)

                           Member  \
0                 Wong, Pak-Heung   
1  Tam, Maria W.C. (Ms) 譚惠珠  (女士)   
2              Necholas, John 黎約翰   
3               Mok, Yeuk-Chi 莫若志   
4              Yuen, Henry M. 袁文超   

                                             Address    Tel.No    Fax.No  \
0                 3/F, 50 Wyndham St,  Central, H.K.  25233450         -   
1  Unit 1203, 12/F, Bank of America Tower, 12 Har...  28698399  25378363   
2  15/F, Printing House, 6 Duddell St, Central, H.K.  25250221  28452441   
3  Rms 45-46, New Henry House, 10 Ice House St, C...  25257295  28106239   
4  Rm 3406 & Rm 412, Tower Two, Lippo Centre, 89 ...  25223365  30072998   

                                                Call       Name of Chambers  \
0                            HK( 1966 ), UK( 1966  )                    NaN   
1                            HK( 1974 ), UK( 1972  )      Kingston Chambers   
2                            HK( 1975 ), UK( 1972  )        Garden Chambers   
3   

## Data cleaning

In [13]:
# read raw data
df = pd.read_csv('香港大律师公会_JuniorCounsel_raw.csv', index_col=0)

Add gender

In [14]:
df['Gender'] = df['Member'].apply(lambda x: 'F' if 'Ms' in x else 'M')

Separate Eng and Chi name

In [26]:
def split_chi_name(name):
    chi_name = re.findall(r'[\u4e00-\u9fa5]+',name)
    if chi_name:
        return chi_name[0]
    else:
        return ""

In [27]:
df['Chi Name'] = df['Member'].apply(split_chi_name)

In [28]:
def split_eng_name(name):
    chi_name = split_chi_name(name)
    if chi_name:
        chi_name_idx = name.find(chi_name)
        return name[:chi_name_idx].strip()
    else: 
        return name

In [32]:
# remove (Ms)
df['Eng Name'] = df['Member'].apply(split_eng_name)
df['Eng Name'] = df['Eng Name'].apply(lambda x: x[:x.find('(')] if '(' in x else x)

Split First and Last Name Eng

In [34]:
df['First Name Eng'] = df['Eng Name'].apply(lambda x: x.split(',')[1] if ',' in x else x)
df['Last Name Eng'] = df['Eng Name'].apply(lambda x: x.split(',')[0] if ',' in x else x)

Split First and Last Name Chi

In [35]:
df['First Name Chi'] = df['Chi Name'].apply(lambda x: x[1:] if x else "")
df['Last Name Chi'] = df['Chi Name'].apply(lambda x: x[0] if x else "")

Order the columns

In [37]:
df.columns

Index(['Member', 'Address', 'Tel.No', 'Fax.No', 'Call', 'Name of Chambers',
       'E-mail', 'Mobile', 'Quals', 'Website', 'Pager', 'Category',
       'Organization', 'Title', 'Gender', 'Chi Name', 'Eng Name',
       'First Name Eng', 'Last Name Eng', 'First Name Chi', 'Last Name Chi'],
      dtype='object')

In [38]:
df = df[['Last Name Chi','First Name Chi','Chi Name',
         'Last Name Eng','First Name Eng','Eng Name','Member',
         'Gender','Title','Category','Organization',
         'Name of Chambers','Quals','Call',
         'Address','Mobile','Tel.No','Fax.No','E-mail','Website','Pager']]

Save to csv file

In [39]:
df.to_excel('香港大律师公会_JuniorCounsel.xlsx',encoding='utf_8_sig')
df.to_csv('香港大律师公会_JuniorCounsel.csv',encoding='utf_8_sig')